In [22]:
# ============================================
# ЯЧЕЙКА 1: КОНВЕРТАЦИЯ КАРТЫ КРАСНОДАРА
# ============================================

import os
import subprocess
from pathlib import Path
import xml.etree.ElementTree as ET
import zipfile

print("="*70)
print("КОНВЕРТАЦИЯ КАРТЫ КРАСНОДАРА ДЛЯ SUMO")
print("="*70)

# Пути
project_root = Path.cwd()
data_dir = project_root / "smart_city" / "data" / "sumo"
osm_file = data_dir / "krasnodar_map.osm"
net_file = data_dir / "krasnodar.net.xml"
sumo_cfg_file = data_dir / "krasnodar.sumocfg"
routes_file = data_dir / "krasnodar_routes.rou.xml"

# Проверяем наличие OSM файла
if not osm_file.exists():
    print(f"❌ Файл карты не найден: {osm_file}")
    print("Скачайте карту Краснодара с OpenStreetMap и сохраните как krasnodar_map.osm")
else:
    print(f"✅ OSM файл найден: {osm_file}")
    size_mb = osm_file.stat().st_size / (1024 * 1024)
    print(f"   Размер: {size_mb:.1f} МБ")

# Проверяем наличие SUMO
SUMO_HOME = os.environ.get('SUMO_HOME', 'C:/Program Files (x86)/Eclipse/Sumo')
NETCONVERT = os.path.join(SUMO_HOME, 'bin', 'netconvert.exe')

if not os.path.exists(NETCONVERT):
    print(f"❌ netconvert.exe не найден: {NETCONVERT}")
else:
    print(f"✅ SUMO найден: {NETCONVERT}")

# Конвертируем OSM в сеть SUMO
print(f"\n🔄 Конвертация OSM в сеть SUMO...")
cmd = [
    NETCONVERT,
    "--osm-files", str(osm_file),
    "-o", str(net_file),
    "--type-files", os.path.join(SUMO_HOME, 'data', 'typemap', 'osmNetconvert.typ.xml'),
    "--geometry.remove", "true",
    "--ramps.guess", "true",
    "--roundabouts.guess", "true",
    "--junctions.join", "true",
    "--tls.guess-signals", "true",
    "--tls.discard-simple", "true",
    "--tls.join", "true",
    "--keep-edges.by-vclass", "passenger,bus,truck,bicycle",
    "--output.original-names", "true",
    "--output.street-names", "true",
    "--proj.plain-geo", "true"
]

try:
    result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
    if result.returncode == 0:
        print(f"✅ Сеть создана: {net_file}")
        
        # Читаем статистику из вывода
        stats = {}
        for line in result.stdout.split('\n'):
            if 'nodes:' in line:
                stats['nodes'] = line.split('nodes:')[1].strip()
            elif 'edges:' in line:
                stats['edges'] = line.split('edges:')[1].strip()
            elif 'tls:' in line:
                stats['tls'] = line.split('tls:')[1].strip()
        
        print(f"📊 Статистика сети:")
        for key, value in stats.items():
            print(f"   {key}: {value}")
        
        # Проверяем наличие светофоров
        if net_file.exists():
            try:
                tree = ET.parse(net_file)
                root = tree.getroot()
                tlss = root.findall('.//tlLogic')
                print(f"   Найдено светофоров в файле: {len(tlss)}")
                
                # Сохраняем список светофоров
                tls_ids = [tls.get('id') for tls in tlss]
                if tls_ids:
                    print(f"   Первые 5 светофоров: {tls_ids[:5]}")
            except Exception as e:
                print(f"⚠️ Ошибка анализа сети: {e}")
    else:
        print(f"❌ Ошибка конвертации: {result.stderr}")
        
except subprocess.TimeoutExpired:
    print("⏱️ Конвертация заняла слишком много времени")
except Exception as e:
    print(f"❌ Ошибка: {e}")

# Создаем базовый конфигурационный файл
print(f"\n📝 Создание конфигурационного файла...")

cfg_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<configuration xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/sumoConfiguration.xsd">
    <input>
        <net-file value="{net_file.name}"/>
        <route-files value="{routes_file.name}"/>
    </input>
    
    <time>
        <begin value="0"/>
        <end value="3600"/>
        <step-length value="1"/>
    </time>
    
    <processing>
        <ignore-route-errors value="true"/>
        <time-to-teleport value="300"/>
        <time-to-teleport.highways value="60"/>
        <lateral-resolution value="0.5"/>
    </processing>
    
    <report>
        <verbose value="true"/>
        <no-step-log value="true"/>
        <no-warnings value="false"/>
    </report>
    
    <gui_only>
        <gui-settings-file value="krasnodar_viewsettings.xml"/>
        <delay value="100"/>
        <start value="true"/>
    </gui_only>
</configuration>"""

with open(sumo_cfg_file, 'w', encoding='utf-8') as f:
    f.write(cfg_content)

print(f"✅ Конфигурационный файл создан: {sumo_cfg_file}")

# Создаем простые маршруты для теста
print(f"\n🗺️ Создание тестовых маршрутов...")

# Сначала получим список ребер из сети
edges = []
if net_file.exists():
    try:
        tree = ET.parse(net_file)
        root = tree.getroot()
        edge_elements = root.findall('.//edge')
        edges = [edge.get('id') for edge in edge_elements if edge.get('id') and not edge.get('id').startswith(':')]
        print(f"   Найдено {len(edges)} ребер")
    except Exception as e:
        print(f"⚠️ Не удалось прочитать сеть: {e}")
        edges = ["-123456789#0", "-123456789#1"]  # Заглушки

# Создаем маршруты
if len(edges) > 3:
    route_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">
    
    <!-- Типы транспортных средств -->
    <vType id="passenger" vClass="passenger" accel="2.6" decel="4.5" sigma="0.5" length="5.0" minGap="2.5" maxSpeed="50" guiShape="passenger"/>
    <vType id="bus" vClass="bus" accel="1.2" decel="3.0" sigma="0.5" length="12.0" minGap="3.0" maxSpeed="40" guiShape="bus"/>
    <vType id="truck" vClass="truck" accel="1.0" decel="2.5" sigma="0.5" length="16.0" minGap="3.0" maxSpeed="35" guiShape="truck"/>
    
    <!-- Базовые маршруты через сеть -->
    <route id="route0" edges="{edges[0]} {edges[1]} {edges[2]}"/>
    <route id="route1" edges="{edges[3]} {edges[4]} {edges[5]}"/>
    <route id="route2" edges="{edges[6]} {edges[7]} {edges[8]}"/>
    <route id="route3" edges="{edges[9]} {edges[10]} {edges[11]}"/>
    
    <!-- Потоки транспортных средств -->
    <flow id="flow0" type="passenger" route="route0" begin="0" end="3600" period="3" departLane="best" departSpeed="max"/>
    <flow id="flow1" type="passenger" route="route1" begin="0" end="3600" period="5" departLane="best" departSpeed="max"/>
    <flow id="flow2" type="bus" route="route2" begin="0" end="3600" period="30" departLane="best" departSpeed="max"/>
    <flow id="flow3" type="truck" route="route3" begin="0" end="3600" period="20" departLane="best" departSpeed="max"/>
    
</routes>"""
else:
    route_content = """<?xml version="1.0" encoding="UTF-8"?>
<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">
    
    <vType id="passenger" vClass="passenger" accel="2.6" decel="4.5" sigma="0.5" length="5.0" minGap="2.5" maxSpeed="50"/>
    
    <route id="route0" edges="edge1 edge2 edge3"/>
    
    <flow id="flow0" type="passenger" route="route0" begin="0" end="3600" period="5"/>
    
</routes>"""

with open(routes_file, 'w', encoding='utf-8') as f:
    f.write(route_content)

print(f"✅ Файл маршрутов создан: {routes_file}")

# Создаем настройки отображения
viewsettings_file = data_dir / "krasnodar_viewsettings.xml"
viewsettings_content = '''<?xml version="1.0" encoding="UTF-8"?>
<viewsettings xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/viewSettingsFile.xsd">
    <scheme name="krasnodar">
        <background backgroundColor="240,240,240"/>
        <lane colorRoad="100,100,100" colorRailway="150,150,150"/>
        <vehicle colorCar="50,100,200" colorBus="220,50,50" colorTruck="34,139,34"/>
        <junction color="80,80,80"/>
        <poi color="150,100,50"/>
        <poly color="200,200,150"/>
        <tlLogic color="255,200,0"/>
        <detector color="255,165,0"/>
        <selection color="255,255,0,128"/>
    </scheme>
    
    <delay value="100"/>
    
    <viewport zoom="100" x="0" y="0"/>
    
    <edges value="true">
        <showLaneDirection value="false"/>
        <showLinkRules value="true"/>
        <widthExaggeration value="1.5"/>
    </edges>
    
    <vehicles value="true">
        <size value="2.0"/>
        <showBlinker value="true"/>
        <drawCarriages value="true"/>
    </vehicles>
</viewsettings>'''

with open(viewsettings_file, 'w', encoding='utf-8') as f:
    f.write(viewsettings_content)

print(f"✅ Настройки отображения созданы: {viewsettings_file}")

print("\n" + "="*70)
print("ГОТОВО! Файлы для карты Краснодара созданы.")
print("="*70)

КОНВЕРТАЦИЯ КАРТЫ КРАСНОДАРА ДЛЯ SUMO
✅ OSM файл найден: C:\Users\ksuzh\my_project_v1\my_project\smart_city\data\sumo\krasnodar_map.osm
   Размер: 7.3 МБ
✅ SUMO найден: C:\Program Files (x86)\Eclipse\Sumo\bin\netconvert.exe

🔄 Конвертация OSM в сеть SUMO...


Exception in thread Thread-17 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\ksuzh\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\ksuzh\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\ksuzh\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ksuzh\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1597, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\ksuzh\AppData\Local\Programs\Python\Python311\Lib\encodings\cp1251.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position

✅ Сеть создана: C:\Users\ksuzh\my_project_v1\my_project\smart_city\data\sumo\krasnodar.net.xml
📊 Статистика сети:
   Найдено светофоров в файле: 38
   Первые 5 светофоров: ['2013503957', '295155284', '295155459', '295155581', '295166005']

📝 Создание конфигурационного файла...
✅ Конфигурационный файл создан: C:\Users\ksuzh\my_project_v1\my_project\smart_city\data\sumo\krasnodar.sumocfg

🗺️ Создание тестовых маршрутов...
   Найдено 976 ребер
✅ Файл маршрутов создан: C:\Users\ksuzh\my_project_v1\my_project\smart_city\data\sumo\krasnodar_routes.rou.xml
✅ Настройки отображения созданы: C:\Users\ksuzh\my_project_v1\my_project\smart_city\data\sumo\krasnodar_viewsettings.xml

ГОТОВО! Файлы для карты Краснодара созданы.
